# MAPF with Abstraction

First, we are going to create the instance, we are trying to solve.

We decided to go for a simple grid with the variable xsize and ysize.
We are at this point constrained to even x and y axis, which could be optimized in the future.

Also you can change the agents' location and their goals.


In [1]:
%%file original_instance.lp
#program base.

#const xsize = 4.
#const ysize = 4.

init(object(raw_node, (X,Y)), value(at, (X,Y))) :- X=1..xsize, Y=1..ysize.

init(object(raw_robot,1),value(at,(1,4))).
init(object(raw_robot,2),value(at,(4,4))).

init(object(raw_shelf,1),value(at,(1,1))).
init(object(raw_shelf,2),value(at,(4,1))).

init(object(raw_product,1),value(on,(1,1))).
init(object(raw_product,2),value(on,(2,1))).

init(object(raw_pickingStation,1),value(at,(2,1))).

init(object(raw_order,1),value(line,(1/3,1))).
init(object(raw_order,1),value(pickingStation,1)).
init(object(raw_order,2),value(line,(2,1))).
init(object(raw_order,2),value(pickingStation,1)).

Overwriting original_instance.lp


## Abstracting the instance

Next, we are going to abstract the instance with this file. 
The logic loosely translates to: 'merge every 4 cells into one cell'

In [2]:
%%file abstracter.lp


% this line creates the abstracted nodes (works with any even xsize and ysize)
init(object(node, (X2/2,Y2/2)), value(at, (X2/2,Y2/2))) :- init(object(raw_node, (X1,Y1)), value(at, (X1,Y1))), init(object(raw_node, (X1,Y2)), value(at, (X1,Y2))), init(object(raw_node, (X2,Y1)), value(at, (X2,Y1))), init(object(raw_node, (X2,Y2)), value(at, (X2,Y2))), X2>X1, Y2>Y1.

% this creates the new abstract robots positions
init(object(robot,N),value(at,((X+1)/2,(Y+1)/2))) :- init(object(raw_robot,N),value(at,(X,Y))).

% this create the rest of the entites
init(object(shelf,N),value(at,((X+1)/2,(Y+1)/2))) :- init(object(raw_shelf,N),value(at,(X,Y))).
init(object(product,N),value(at,((X+1)/2,(Y+1)/2))) :- init(object(raw_product,N),value(at,(X,Y))).
init(object(pickingStation,N),value(at,((X+1)/2,(Y+1)/2))) :- init(object(raw_pickingStation,N),value(at,(X,Y))).
init(object(order,N),value(at,((X+1)/2,(Y+1)/2))) :- init(object(raw_order,N),value(at,(X,Y))).
    
%maybe try to only show the atoms, that dont have raw in them 
%#show init/2.

Overwriting abstracter.lp


Running this with the clingo, gives us the abstracted instance.

There were problems with how to export this abstracted instance, so that the solver can use it.

The following commands creates the file `abstracted_instance.lp`, which is the final abstracted instance.
Please note, that the raw atoms of the instance are still in there.

In [11]:
!rm abstracted_instance_raw.lp
!clingo abstracter.lp original_instance.lp --quiet=1 --opt-mode=OptN --out-ifs=\\n >> abstracted_instance_raw.lp

In [12]:
!rm abstracted_instance_clean.lp
!head -n $(( $(wc -l abstracted_instance_raw.lp | awk '{print $1}') - 6 )) abstracted_instance_raw.lp  >> abstracted_instance_clean.lp

In [13]:
!rm abstracted_instance_cleanest.lp
!tail -n $(( $(wc -l abstracted_instance_clean.lp | awk '{print $1}') - 4 )) abstracted_instance_clean.lp  >> abstracted_instance_cleanest.lp


In [52]:
!rm abstracted_instance.lp
!cat abstracted_instance_cleanest.lp | while read line; do echo ${line}.; done >> abstracted_instance.lp
!cat abstracted_instance.lp

init(object(raw_order,1),value(pickingStation,1)).
init(object(raw_order,2),value(pickingStation,1)).
init(object(raw_robot,1),value(at,(1,4))).
init(object(raw_robot,2),value(at,(4,4))).
init(object(raw_shelf,1),value(at,(1,1))).
init(object(raw_shelf,2),value(at,(4,1))).
init(object(raw_product,1),value(on,(1,1))).
init(object(raw_product,2),value(on,(2,1))).
init(object(raw_pickingStation,1),value(at,(2,1))).
init(object(raw_order,1),value(line,(0,1))).
init(object(raw_order,2),value(line,(2,1))).
init(object(pickingStation,1),value(at,(1,1))).
init(object(shelf,1),value(at,(1,1))).
init(object(shelf,2),value(at,(2,1))).
init(object(robot,1),value(at,(1,2))).
init(object(robot,2),value(at,(2,2))).
init(object(raw_node,(1,1)),value(at,(1,1))).
init(object(raw_node,(2,1)),value(at,(2,1))).
init(object(raw_node,(3,1)),value(at,(3,1))).
init(object(raw_node,(4,1)),value(at,(4,1))).
init(object(raw_node,(1,2)),value(at,(1,2))).
init(object(raw_node,(2,2)),value(at,(2,2))).
init(object(ra

Exectute the following in a terminal to start the solver

`cd m`

`viz-solver --port 5001`

Then, in another terminal run:

`viz -t abstracted_instance.lp`

This should open the visualisation of the instances. Then on `Network` choose `solve` and change the port to `5001`.
After some time, the solution should appear on the right side.
It should consist of occurs() statements and you can save that plan with `File>save plan` and save the plan as `abstracted_solution.lp`.

In case that didn't work, here is the solution for the first example instance.

In [15]:
%%file abstracted_solution.lp
occurs(object(robot, 1), action(move,(0,-1)), 1).
occurs(object(robot, 1), action(move,(0,-1)), 2).  
occurs(object(robot, 1), action(move,(1,0)), 3).
occurs(object(robot, 1), action(move,(1,0)), 4). 
occurs(object(robot, 1), action(move,(1,0)), 5). 

occurs(object(robot, 2), action(move,(0,-1)), 1).
occurs(object(robot, 2), action(move,(0,-1)), 2).

Overwriting abstracted_solution.lp


## De-abstracting the solution
We create the `deabstracter.lp` to map the abstracted solution to a solution in the original map dimensions.
The basic idea is to add an extra step in the same direction for every move made in the abstracted solution. This corresponds to the fact that for every grid cell a robot moves in the abstracted map it has to move twice in the original map. 

In [8]:
%%file deabstracter.lp

% find max timestep
t(T) :- occurs(object(robot, ID), action(move,(X,Y)), T).
max_t(T) :- t(T), #max{TT, 1:t(TT)}=T.

% assume goal is reachable by changing direction max 1 time
% mark the first and second direction for each robot
first((X, Y), ID) :- occurs(object(robot, ID), action(move,(X,Y)), 1).
second((X2, Y2), ID) :- occurs(object(robot, ID), action(move,(X2,Y2)), T), max_t(T), first((X,Y), ID2), ID !=ID2.
    
% encode moves into single integer to be able to count them
move(1,ID, T):-  occurs(object(robot, ID), action(move,(X,Y)), T), X=0, Y=-1.
move(2,ID, T):-  occurs(object(robot, ID), action(move,(X,Y)), T), X=1, Y=0.
move(3,ID, T):-  occurs(object(robot, ID), action(move,(X,Y)), T), X=-1, Y=0.
move(4,ID, T):-  occurs(object(robot, ID), action(move,(X,Y)), T), X=0, Y=1.

% count repetitions of each move for each robot
count((0,-1),ID, N) :- move(1,ID, _), N = #count{move(1,ID, T): occurs(object(robot, ID), action(move,(0,-1)), T) }, N>0.
count((1, 0),ID, N) :- move(1,ID, _), N = #count{move(1,ID, T): occurs(object(robot, ID), action(move,(1,0)), T) }, N>0.
count((-1, 0),ID, N) :- move(1,ID, _), N = #count{move(1,ID, T): occurs(object(robot, ID), action(move,(-1,0)), T) }, N>0.
count((0, 1),ID, N) :- move(1,ID, _), N = #count{move(1,ID, T): occurs(object(robot, ID), action(move,(0,1)), T) }, N>0.

% set the target number of moves in deabstracted solution
goal((X, Y), ID, N*2) :- count((X, Y), ID, N).

% set the range of time-steps for each move for each robot
range((X,Y), ID, (1, M)) :- first((X,Y), ID), goal((X, Y), ID, M).
range((X2,Y2), ID, (M+1, M+M2)) :- second((X2,Y2), ID), goal((X2, Y2), ID, M2), first((X,Y), _), goal((X, Y),_, M).

%finally generate the motion commands based on the range set
new_occurs(object(robot, ID), action(move, (X, Y)), A) :- range((X,Y), ID, (A, _)).
new_occurs(object(robot, ID), action(move, (X, Y)), T+1)  :- new_occurs(object(robot, ID), action(move, (X, Y)), T), range((X,Y), ID, (_, B)), T<B.

Overwriting deabstracter.lp


### Filter out the irrelevant atoms
Clingo generates atoms of the form `new_occurs(...)` aditionally to the original `occurs(...)` atoms. With the the `awk '/new/ {{print}}'` command we filter out everything except the relevant atoms of the new plan, that is the `new_occurs(...)`. 

In [49]:
!rm -f deabstracted_solution_raw.lp
!clingo deabstracter.lp abstracted_solution.lp --quiet=1 --opt-mode=OptN --out-ifs=\\n | awk '/new/ {{print}}' >> deabstracted_solution_raw.lp
!cat deabstracted_solution_raw.lp

deabstracter.lp:18:57-59: info: global variable in tuple of aggregate element:
  ID

deabstracter.lp:19:57-59: info: global variable in tuple of aggregate element:
  ID

deabstracter.lp:20:58-60: info: global variable in tuple of aggregate element:
  ID

deabstracter.lp:21:57-59: info: global variable in tuple of aggregate element:
  ID

new_occurs(object(robot,1),action(move,(0,-1)),1)
new_occurs(object(robot,2),action(move,(0,-1)),1)
new_occurs(object(robot,1),action(move,(1,0)),5)
new_occurs(object(robot,1),action(move,(1,0)),6)
new_occurs(object(robot,2),action(move,(0,-1)),2)
new_occurs(object(robot,1),action(move,(0,-1)),2)
new_occurs(object(robot,1),action(move,(0,-1)),3)
new_occurs(object(robot,2),action(move,(0,-1)),3)
new_occurs(object(robot,1),action(move,(1,0)),7)
new_occurs(object(robot,1),action(move,(1,0)),8)
new_occurs(object(robot,2),action(move,(0,-1)),4)
new_occurs(object(robot,1),action(move,(0,-1)),4)
new_occurs(object(robot,1),action(move,(1,0)),9)
new_occurs(obje

### Rename atoms in solution file back to original format
We then rename the `new_occurs(...)` to the original `occurs(..)` format. This `deabstracted_solution.lp` file can then be loaded in the vizualisation tool as a plan to solve the original instance. Note that the search for a solution only happened in the abstracted instance.

In [50]:
!rm -rf deabstracted_solution_clean.lp
!sed 's/new_//g' deabstracted_solution_raw.lp >> deabstracted_solution_clean.lp
!cat deabstracted_solution_clean.lp

occurs(object(robot,1),action(move,(0,-1)),1)
occurs(object(robot,2),action(move,(0,-1)),1)
occurs(object(robot,1),action(move,(1,0)),5)
occurs(object(robot,1),action(move,(1,0)),6)
occurs(object(robot,2),action(move,(0,-1)),2)
occurs(object(robot,1),action(move,(0,-1)),2)
occurs(object(robot,1),action(move,(0,-1)),3)
occurs(object(robot,2),action(move,(0,-1)),3)
occurs(object(robot,1),action(move,(1,0)),7)
occurs(object(robot,1),action(move,(1,0)),8)
occurs(object(robot,2),action(move,(0,-1)),4)
occurs(object(robot,1),action(move,(0,-1)),4)
occurs(object(robot,1),action(move,(1,0)),9)
occurs(object(robot,1),action(move,(1,0)),10)
